# Making embeddings from tg channels

### preparing  data

In [1]:
# importing libraries
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
import torch
from transformers import AutoTokenizer, AutoModel
from typing import List
torch.cuda.is_available()

In [2]:
#raw parsed texts data
files = os.listdir()[2:-1]

In [3]:
# uploading files to DataFrame
data = pd.DataFrame()
for i in tqdm(files):
    data = pd.concat([data, pd.read_json(i)])
data = data.reset_index(drop=True)

100%|██████████| 12/12 [00:01<00:00,  9.00it/s]


In [4]:
mes = data['messages'].values

In [6]:
# extracting texts
texts = []
for i in tqdm(mes):
    try:
        mes_str = i['text']
    except KeyError:
        pass
    
    if type(mes_str) == list:
        if type(mes_str[0]) != dict:
            if mes_str[0] != '':
                texts.append(mes_str[0])

100%|██████████| 70687/70687 [00:00<00:00, 1570470.09it/s]


In [7]:
#all texts len
len(texts)

45545

In [8]:
#deleting garbage chars
for i in range(0, len(texts)):
    texts[i] = texts[i].replace('\n', '')

In [9]:
#saving raw texts
with open('texts_cb.pkl', 'wb') as f:
    pickle.dump(texts, f)

### bert encoding

In [12]:
#model downloading
device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [13]:
#encode function
def encode(text:List):
    encoding = tokenizer(text, return_tensors='pt', padding=True, max_length=512, truncation=True).to(device)
    return model(**encoding).pooler_output.cpu().detach().numpy()

In [14]:
#countiong max_len of tg posts
length = []

for i in range(0, len(texts)):
    length.append(len(texts[i].split()))


In [15]:
#max_len
np.array(length).max()

609

In [16]:
#model to cuda
model = model.to(device)

In [27]:
!nvidia-smi

Sun Sep 24 00:26:21 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.13                 Driver Version: 537.13       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  | 00000000:01:00.0  On |                  Off |
|  0%   32C    P8              28W / 450W |  14717MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [18]:
#computing embeddings
X_all = np.zeros([1,1024])

for i in tqdm(range(0, len(texts), 5)):
    
    X_all = np.append(X_all, encode(list(texts[i:i+5])), axis=0)

100%|██████████| 9109/9109 [06:39<00:00, 22.81it/s]


In [19]:
#deletting zeros array 
X_all = X_all[1:,:]

In [20]:
#shape of embeddings
X_all.shape

(45545, 1024)

In [21]:
# saving embeddings
with open('texts_encoded.pkl', 'wb') as f:
    pickle.dump(X_all, f)